In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd
import random
import os

In [2]:
version = "1.0.1"
datadir = "data/"
archivedir = "data/Archive"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(filename):
    # open old file
    olddf = pd.DataFrame({'url': [] })
    try:
        olddf = pd.read_pickle(os.path.join(archivedir, filename))
        print(olddf.tail())
    except:
        pass

    driver.get("https://www.combinatorics.org/ojs/index.php/eljc/issue/archive")
    
    # get all of the issue
    def issues():
        issues = []
        nexpage = False
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for div in soup.find_all('div', {'class': 'obj_issue_summary'}):
            for a in div.find_all('a'):
                # if it is this journal
                if re.search("Volume", " ".join([ str(x) for x in a.contents ]) ):
                    issues.append(a['href'])
        # check there is new issue
        for a in soup.find_all('a', {'class': 'next'}):
            nexpage = True
            # set to nexpage
            driver.get(a['href'])
            sleep(3)
        
        return  issues, nexpage
    
    nextpage = True
    allissue = []
    while  nextpage:
        thispageissues, nextpage = issues()
        allissue = allissue + thispageissues
        print(driver.current_url, end='\r') 
    allissue = list(set(allissue))
    
    # iterate over isses and get article links
    articellinks = []
    for issue in allissue:
        driver.get(issue)
        sleep(2)
        print(driver.current_url, end='\r') 
        # get the issue article
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for div in soup.find_all('div', {'class': 'obj_article_summary'}):
            for title in div.find_all('div', {'class': 'title'}):
                for a in title.find_all("a"):
                    articellinks.append(a['href'])        
    articellinks = list(set(articellinks))
    
    # filer out old articles
    oldarticles = olddf['url']
    articellinks = list( set(articellinks).difference(set(oldarticles) ))
    print(len(articellinks), "new article found!")
    
    # iterate over article 
    urls = [ None for _ in range(len(articellinks))]
    titles = [ None for _ in range(len(articellinks))]
    abstracts = [ None for _ in range(len(articellinks))]
    writers = [ None for _ in range(len(articellinks))]
    dates = [ None for _ in range(len(articellinks))]
    dois = [  None for _ in range(len(articellinks)) ]
    keywords = [  None for _ in range(len(articellinks)) ]
    for idx in range(len(articellinks)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(articellinks),2))+"%" , end='\r')
        url =  articellinks[idx]
        urls[idx] = url
        # load page
        driver.get(articellinks[idx])
        sleep(2)
        # get the issue article
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # process metadata
        keyword = []
        writter = []
        notpaper = False
        for m in soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == 'DC.Title':
                    titles[idx] = m['content']
                if m['name'] == 'DC.Type.articleType':
                    if  m['content'].strip() != 'Papers' and  m['content'].strip() != 'Research Papers' and \
                    m['content'].strip() != 'Articles' :
                        print("Not a Papers", url)
                        notpaper = True
                if m['name'] == "citation_keywords":
                    keyword.append( m['content'] )
                if m['name'] == "DC.Date.issued":
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y-%m-%d").strftime('%Y-%m-%d')
                if m['name'] == 'DC.Creator.PersonalName':
                    writter.append(m['content'])
                if m['name'] == 'DC.Identifier.DOI' or m['name'] == 'citation_doi':
                    dois[idx] = m['content']                    
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        if len(keyword) > 0:
            keywords[idx] =  "#".join(keyword)
            
        # get abstract
        abstract = ""
        if notpaper:
            continue
        else:
            for div in soup.find_all('div', {'class': 'abstract'}):
                for c in div.find_all(['p', 'script']):
                    if c.name == "p":                    
                        for tag in  c.contents:
                            if re.search("<span", str(tag)):
                                continue
                            else:
                                match = re.match("<script.+>", str(tag))
                                if match is not None:
                                    abstract = abstract + " $"+str(tag).split('>')[1].split('<')[0] + "$ "
                                else:
                                    abstract = abstract + str(tag).replace("\n", '').replace("\t", '')
                # if still not abstract it is possible it is not in p
                if len(abstract) == 0:
                    for tag in div.contents:
                        if re.search("<span", str(tag)) or re.search("<h3", str(tag)) :
                            continue
                        else:
                            match = re.match("<script.+>", str(tag))
                            if match is not None:
                                abstract = abstract + " $"+str(tag).split('>')[1].split('<')[0] + "$ "
                            else:
                                abstract = abstract + str(tag).replace("\n", '').replace("\t", '')
            abstracts[idx] = abstract        
    
    # make df
    df = pd.DataFrame({'url': urls, 
                       'journal_title': "Electronic Journal of Combinatorics", 
                       'journal_eissn': "1077-8926",
                       'journal_pissn': '',
                       'category': "Mathematics",                                              
                      })
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # drop where is no abstract
    df = df[ df['abstract'].notna()]
    
    # check there was a not Arhived but previously loaded file
    adf = None
    try:
        adf = pd.read_pickle(os.path.join(datadir, filename))
    except:
        pass
    if adf is not None:
        df = pd.concat([df, adf])
        df.drop_duplicates(inplace=True)
    
    return df, olddf

In [5]:
filename = 'journal_Electronic_Journal_of_Combinatorics_'+version+'.pandas'
    
# search for articles
df, olddf = gedjournalarticles( filename )

# save data
df.to_pickle(os.path.join(datadir, filename))

print(df.head())


4016 new article found!cs.org/ojs/index.php/eljc/issue/view/Volume10-1
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1n4
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v10i1n12
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v14i1n8
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v13i1n18
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n19
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n7
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n33
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v10i1n1
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v8i2i3
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n21
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n40
Not a Paper

Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n23
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n28
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1n12
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1n29
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v13i1n4
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v12i1n8
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n13
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v14i1n17
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n18
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v9i1n3
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1n5
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v14i1n

Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n16
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1n36
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n23
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v13i1n10
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v7i1n5
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n30
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n27
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n24
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v8i1n7
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v16i1n31
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v15i1n17
Not a Papers https://www.combinatorics.org/ojs/index.php/eljc/article/view/v17i1

In [6]:
print(df.iloc[0]['abstract'])

Using methods developed by Coons and Tyler, we give a new proof of a recent result of Defant, by determining the maximal order of the number of hyper-( $b$ -ary)-expansions of a nonnegative integer  $n$  for general integral bases  $b\geqslant 2$ .


In [7]:
print(df.iloc[0])

url              https://www.combinatorics.org/ojs/index.php/el...
journal_title                  Electronic Journal of Combinatorics
journal_eissn                                            1077-8926
journal_pissn                                                     
category                                               Mathematics
title              The Maximal Order of Hyper-($b$-ary)-expansions
doi                                                  10.37236/5441
abstract         Using methods developed by Coons and Tyler, we...
writer                            Michael Coons#Lukas Spiegelhofer
publishdate                                             2017-01-20
keyword          Stern's diatomic sequence#Hyper base expansion...
Name: 0, dtype: object


In [8]:
driver.close()